## Preprocess Weather Data (.csv)

In [1]:
# Import required packages
import pandas as pd
import os

In [2]:
# Read in the weather dataset
weather = pd.read_csv("../data/raw/nyc_weather_raw.csv")

/var/folders/bz/yjywszk15kxc8d5vv73k9xcc0000gn/T/ipykernel_25863/3729253711.py:2: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123) have mixed types. Specify dtype option on import or set low_memory=False.
  weather = pd.read_csv("../data/raw/nyc_weather_raw.csv")


In [3]:
weather.count()

STATION            56833
DATE               56833
LATITUDE           56833
LONGITUDE          56833
ELEVATION          56833
                   ...  
WT19_ATTRIBUTES      179
WT21                   1
WT21_ATTRIBUTES        1
WT22                  85
WT22_ATTRIBUTES       85
Length: 124, dtype: int64

In [4]:
weather.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,...,WT17,WT17_ATTRIBUTES,WT18,WT18_ATTRIBUTES,WT19,WT19_ATTRIBUTES,WT21,WT21_ATTRIBUTES,WT22,WT22_ATTRIBUTES
0,USW00094728,1869-01-01,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",191,",,Z,",229.0,",,Z,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00094728,1869-01-02,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",8,",,Z,null",0.0,",,Z,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00094728,1869-01-03,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",0,"T,,Z,null",0.0,",,Z,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00094728,1869-01-04,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",46,",,Z,null",0.0,",,Z,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00094728,1869-01-05,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",13,",,Z,null",0.0,",,Z,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# # Identify and remove _ATTRIBUTES columns
# columns_to_drop = [col for col in weather.columns if '_ATTRIBUTES' in col]
# weather.drop(columns=columns_to_drop, inplace=True)

# Columns to keep (excluding the _ATTRIBUTES columns)
columns_to_keep = [
    'DATE', 'PRCP', 'SNOW', 'TMAX', 'TMIN', 'AWND', 
    'RHAV', 'WT01', 'WT02', 'WT03'
]

# Keep only the specified columns
weather = weather[columns_to_keep]

The knowledge of what the column codes stand for comes from https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt

In [6]:
# Rename columns for clarity
rename_columns = {
    'DATE': 'date',
    'PRCP': 'precip',
    'SNOW': 'snowfall',
    'TMAX': 'max_temp',
    'TMIN': 'min_temp',
    'AWND': 'avg_wind_sp',
    'RHAV': 'avg_rel_humi',
    'WT01': 'fog',
    'WT02': 'heavy_fog',
    'WT03': 'thunder'
}

weather.rename(columns=rename_columns, inplace=True)

In [7]:
weather.head()

,date,precip,snowfall,max_temp,min_temp,avg_wind_sp,avg_rel_humi,fog,heavy_fog,thunder
0,1869-01-01,191,229.0,-17.0,-72.0,NaN,NaN,NaN,NaN,NaN
1,1869-01-02,8,0.0,-28.0,-61.0,NaN,NaN,NaN,NaN,NaN
2,1869-01-03,0,0.0,17.0,-28.0,NaN,NaN,NaN,NaN,NaN
3,1869-01-04,46,0.0,28.0,11.0,NaN,NaN,NaN,NaN,NaN
4,1869-01-05,13,0.0,61.0,28.0,NaN,NaN,NaN,NaN,NaN


In [8]:
# Filter rows by date range
weather['date'] = pd.to_datetime(weather['date'])
start_date = '2023-01-01'
end_date = '2024-04-30'
weather = weather[(weather['date'] >= start_date) & (weather['date'] <= end_date)]

In [9]:
weather.count()

date            486
precip          486
snowfall        486
max_temp        486
min_temp        486
avg_wind_sp     484
avg_rel_humi    454
fog             186
heavy_fog        11
thunder          36
dtype: int64

#### Handling the NaN values

In [10]:
# Replace NaN in 'avg_wind_sp' and 'avg_rel_humi' with the mean of neighboring values
weather['avg_wind_sp'] = weather['avg_wind_sp'].interpolate(method='linear', limit_direction='both')
weather['avg_rel_humi'] = weather['avg_rel_humi'].interpolate(method='linear', limit_direction='both')

# Replace NaN in columns from 'fog' onwards with 0
columns_to_replace = ['fog', 'heavy_fog', 'thunder']
weather[columns_to_replace] = weather[columns_to_replace].fillna(0)

In [11]:
weather.head()

,date,precip,snowfall,max_temp,min_temp,avg_wind_sp,avg_rel_humi,fog,heavy_fog,thunder
56247,2023-01-01,0,0.0,128.0,94.0,22.0,64.0,1.0,0.0,0.0
56248,2023-01-02,5,0.0,133.0,94.0,16.0,65.0,0.0,0.0,0.0
56249,2023-01-03,107,0.0,144.0,83.0,14.0,89.0,1.0,0.0,0.0
56250,2023-01-04,5,0.0,189.0,94.0,20.0,80.0,1.0,0.0,0.0
56251,2023-01-05,3,0.0,100.0,67.0,23.0,90.0,1.0,0.0,0.0


In [12]:
# Create another column for the average temperature (taken from the minimum and maximum temperature)
weather['avg_temp'] = weather[['max_temp', 'min_temp']].mean(axis=1)

In [13]:
weather.head()

,date,precip,snowfall,max_temp,min_temp,avg_wind_sp,avg_rel_humi,fog,heavy_fog,thunder,avg_temp
56247,2023-01-01,0,0.0,128.0,94.0,22.0,64.0,1.0,0.0,0.0,111.0
56248,2023-01-02,5,0.0,133.0,94.0,16.0,65.0,0.0,0.0,0.0,113.5
56249,2023-01-03,107,0.0,144.0,83.0,14.0,89.0,1.0,0.0,0.0,113.5
56250,2023-01-04,5,0.0,189.0,94.0,20.0,80.0,1.0,0.0,0.0,141.5
56251,2023-01-05,3,0.0,100.0,67.0,23.0,90.0,1.0,0.0,0.0,83.5


In [14]:
# Define the directory path
output_dir = '../data/curated/weather'

# Check if the directory exists, if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [15]:
weather.to_csv("../data/curated/weather/weather_cleaned.csv", index=False)